# Tiny ImageNet: Validation Accuracy

Calculate top-1 and top-5 validation accuracy. 2 methods:

1. Feed validation image into model with no distortions
2. Feed validation image in with 10 permuations of l/r flip and 5 crops

The validation set was used because labels weren't available for the test set.

Note: This runs *very* slowly. The images are loaded one-by-one in a loop and fed to the model using a feed_dict. During training, a much more efficient pipeline is used: distortions and QueueRunner on CPU and model on the GPU.

*Python Notebook by Patrick Coady: [Learning Artificial Intelligence](https://learningai.io/)*

In [1]:
from train import *
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import random
import scipy.ndimage
import scipy.misc

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
class TrainConfig(object):
    """Training configuration"""
    dropout_keep_prob = 1.0  # disable dropout for inference
    model_name = 'vgg_16'  # choose model 
    model = staticmethod(globals()[model_name])
    config_name = 'no_hue'  # choose training run

In [3]:
def accuracy(config):
    """Load most recent checkpoint and make prediction"""
    with tf.Graph().as_default():
        img_ph = tf.placeholder(dtype=tf.uint8, shape=(None, 64, 64, 3))
        img = tf.image.crop_to_bounding_box(img_ph, 4, 4, 56, 56)
        logits = config.model(img, config)      
        top_5 = tf.nn.top_k(logits, k=5, sorted=True)
        saver = tf.train.Saver()
        with tf.Session() as sess:
            filenames_labels = load_filenames_labels('val')
            path = 'checkpoints/' + config.model_name + '/' + config.config_name
            saver.restore(sess, tf.train.latest_checkpoint(path))
            count, correct1, correct5 = (0, 0, 0)
            for filename, label in filenames_labels:
                count += 1
                np_img = scipy.ndimage.imread(filename, mode='RGB')
                np_img = np_img[np.newaxis, :, :, :]
                feed_dict = {img_ph: np_img}
                top_vals, top_idx = sess.run(top_5, feed_dict=feed_dict)
                if top_idx[0][0] == int(label):
                    correct1 += 1
                if int(label) in top_idx[0]:
                    correct5 += 1
                    
      
    return correct1 / count, correct5 / count

In [4]:
def accuracy_10_crop(config):
    """Load most recent checkpoint and make prediction"""
    with tf.Graph().as_default():
        img_ph = tf.placeholder(dtype=tf.uint8, shape=(64, 64, 3))
        crops = [(0,0), (0,8), (8,0), (8,8), (4,4)]
        img_list = []
        for crop in crops:  # 5 crops * 2 flip l-r
            x, y = crop
            img = tf.image.crop_to_bounding_box(img_ph, x, y, 56, 56)
            img_list.append(img)
            img = tf.image.flip_left_right(img)
            img_list.append(img)
        img = tf.stack(img_list)
        ps = tf.nn.softmax(config.model(img, config))
        ps = tf.reduce_prod(ps, axis=0, keep_dims=True) 
        top_k = tf.nn.top_k(ps, k=5, sorted=True)
        saver = tf.train.Saver()
        with tf.Session() as sess:
            filenames_labels = load_filenames_labels('val')
            path = 'checkpoints/' + config.model_name + '/' + config.config_name
            saver.restore(sess, tf.train.latest_checkpoint(path))
            count, correct1, correct5 = (0, 0, 0)
            for filename, label in filenames_labels:
                count += 1
                np_img = scipy.ndimage.imread(filename, mode='RGB')
                feed_dict = {img_ph: np_img}
                top_vals, top_idx = sess.run(top_k, feed_dict=feed_dict)
                if top_idx[0][0] == int(label):
                    correct1 += 1
                if int(label) in top_idx[0]:
                    correct5 += 1
      
    return correct1 / count, correct5 / count

In [5]:
config = TrainConfig()
acc = accuracy(config)
print('Top-1 accuracy (center crop): {}, Top-5 accuracy: {}'.format(acc[0], acc[1]))

INFO:tensorflow:Restoring parameters from checkpoints/vgg_16/no_hue/model-44000
Top-1 accuracy (center crop): 0.5185, Top-5 accuracy: 0.7654


In [6]:
acc = accuracy_10_crop(config)
print('Top-1 accuracy (5-crops + l/r flip): {}, Top-5 accuracy: {}'.format(acc[0], acc[1]))

INFO:tensorflow:Restoring parameters from checkpoints/vgg_16/no_hue/model-44000
Top-1 accuracy (5-crops + l/r flip): 0.5469, Top-5 accuracy: 0.787


## Results Summary

**'baseline' performance**  
Top-1 accuracy (center crop): 0.4929, Top-5 accuracy: 0.7478  
Top-1 accuracy (5-crops + l/r flip): 0.522, Top-5 accuracy: 0.7723  
  
**smoothed cross-entropy**  
Top-1 accuracy (center crop): 0.4958, Top-5 accuracy: 0.7512  
Top-1 accuracy (5-crops + l/r flip): 0.5197, Top-5 accuracy: 0.7695  
  
*There is no significant improvement from using smoothed cross-entropy loss.*

## Ablation with Image Distortions Removed

### No Flip
Top-1 accuracy (center crop): 0.4613, Top-5 accuracy: 0.7156  
Top-1 accuracy (5-crops + l/r flip): 0.5127, Top-5 accuracy: 0.7572  

### No Crop
Top-1 accuracy (center crop): 0.4441, Top-5 accuracy: 0.6986  
Top-1 accuracy (5-crops + l/r flip): 0.4979, Top-5 accuracy: 0.7494  

### No Saturation
Top-1 accuracy (center crop): 0.5254, Top-5 accuracy: 0.7691  
Top-1 accuracy (5-crops + l/r flip): 0.5525, Top-5 accuracy: 0.7895  

### No Hue
Top-1 accuracy (center crop): 0.5286, Top-5 accuracy: 0.771  
Top-1 accuracy (5-crops + l/r flip): 0.5644, Top-5 accuracy: 0.7929  

### No Saturation / No Hue
Top-1 accuracy (center crop): 0.5195, Top-5 accuracy: 0.7621  
Top-1 accuracy (5-crops + l/r flip): 0.5482, Top-5 accuracy: 0.7817  


